# Run fitsverify

In [1]:
import os
import sys
import re
import shutil
import subprocess as sp
from configparser import ConfigParser
from random import choice
specprod = 'everest'
specprod_path = os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod)

## Create input file

In [2]:
fits_files = os.path.join(os.environ['CSCRATCH'], f'{specprod}_fits.txt')
if not os.path.exists(fits_files):
    # os.chdir(specprod_path)
    with open(fits_files, 'w') as out:
        command = ['find', 'calibnight', 'exposures', 'healpix', 'preproc', 'tiles', 'zcatalog', '-type', 'f', '-name', '*.fits']
        proc = sp.Popen(command, stdout=out, stderr=sp.DEVNULL, cwd=specprod_path)
        status = proc.wait()

## List of Regular Expressions

In [3]:
parser = ConfigParser()
parser.read_string("""
[top]
exposures = exposures-everest\.fits;exposures-SPECPROD.rst

[calibnight]
fiberflatnight = calibnight/[0-9]{8}/fiberflatnight-[brz][0-9]-[0-9]{8}\.fits;calibnight/NIGHT/fiberflatnight-CAMERA-EXPID.rst
psfnight = calibnight/[0-9]{8}/psfnight-[brz][0-9]-[0-9]{8}\.fits;calibnight/NIGHT/psfnight-CAMERA-EXPID.rst

[exposures]
cframe = exposures/[0-9]{8}/[0-9]{8}/cframe-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/cframe-CAMERA-EXPID.rst
exposure-qa = exposures/[0-9]{8}/[0-9]{8}/exposure-qa-[0-9]{8}\.fits;exposures/NIGHT/EXPID/exposure-qa-EXPID.rst
fiberflat = exposures/[0-9]{8}/[0-9]{8}/fiberflat-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fiberflat-CAMERA-EXPID.rst
fit-psf = exposures/[0-9]{8}/[0-9]{8}/fit-psf-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-CAMERA-EXPID.rst
fit-psf-before-blacklisted = exposures/[0-9]{8}/[0-9]{8}/fit-psf-before-blacklisted-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-before-blacklisted-CAMERA-EXPID.rst
fit-psf-before-blacklisted-fix = exposures/[0-9]{8}/[0-9]{8}/fit-psf-before-blacklisted-fix-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-before-blacklisted-fix-CAMERA-EXPID.rst
fit-psf-fixed-blacklisted = exposures/[0-9]{8}/[0-9]{8}/fit-psf-fixed-blacklisted-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-fixed-blacklisted-CAMERA-EXPID.rst
fluxcalib = exposures/[0-9]{8}/[0-9]{8}/fluxcalib-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fluxcalib-CAMERA-EXPID.rst
frame = exposures/[0-9]{8}/[0-9]{8}/frame-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/frame-CAMERA-EXPID.rst
psf = exposures/[0-9]{8}/[0-9]{8}/psf-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/psf-CAMERA-EXPID.rst
sframe = exposures/[0-9]{8}/[0-9]{8}/sframe-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/sframe-CAMERA-EXPID.rst
shifted-input-psf = exposures/[0-9]{8}/[0-9]{8}/shifted-input-psf-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/shifted-input-psf-CAMERA-EXPID.rst
sky = exposures/[0-9]{8}/[0-9]{8}/sky-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/sky-CAMERA-EXPID.rst
stdstars = exposures/[0-9]{8}/[0-9]{8}/stdstars-[0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/stdstars-SPECTROGRAPH-EXPID.rst

[healpix]
coadd = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/coadd-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/coadd-SURVEY-PROGRAM-PIXNUM.rst
qso_mgii = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/qso_mgii-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/qso_mgii-SURVEY-PROGRAM-PIXNUM.rst
qso_qn = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/qso_qn-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/qso_qn-SURVEY-PROGRAM-PIXNUM.rst
redrock = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/redrock-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/redrock-SURVEY-PROGRAM-PIXNUM.rst
spectra = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/spectra-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/spectra-SURVEY-PROGRAM-PIXNUM.rst
tilepix = healpix/tilepix\.fits;healpix/tilepix.rst

[preproc]
fibermap = preproc/[0-9]{8}/[0-9]{8}/fibermap-[0-9]{8}\.fits;preproc/NIGHT/EXPID/fibermap-EXPID.rst
preproc = preproc/[0-9]{8}/[0-9]{8}/preproc-[brz][0-9]-[0-9]{8}\.fits;preproc/NIGHT/EXPID/preproc-CAMERA-EXPID.rst

[tiles]
coadd = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/coadd-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/coadd-SPECTROGRAPH-NIGHT-EXPID.rst
qso_mgii = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/qso_mgii-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/qso_mgii-SPECTROGRAPH-NIGHT-EXPID.rst
qso_qn = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/qso_qn-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/qso_qn-SPECTROGRAPH-NIGHT-EXPID.rst
redrock = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/redrock-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/redrock-SPECTROGRAPH-NIGHT-EXPID.rst
spectra = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/spectra-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/spectra-SPECTROGRAPH-NIGHT-EXPID.rst
tile-qa = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/tile-qa-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/tile-qa-NIGHT-EXPID.rst

[tiles:depth]
coadd = tiles/[14]x_depth/[0-9]+/[0-9]/coadd-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/coadd-SPECTROGRAPH-TILEID-EXPID.rst
qso_mgii = tiles/[14]x_depth/[0-9]+/[0-9]/qso_mgii-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/qso_mgii-SPECTROGRAPH-TILEID-EXPID.rst
qso_qn = tiles/[14]x_depth/[0-9]+/[0-9]/qso_qn-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/qso_sn-SPECTROGRAPH-TILEID-EXPID.rst
redrock = tiles/[14]x_depth/[0-9]+/[0-9]/redrock-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/redrock-SPECTROGRAPH-TILEID-EXPID.rst
spectra = tiles/[14]x_depth/[0-9]+/[0-9]/spectra-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/spectra-SPECTROGRAPH-TILEID-EXPID.rst

[zcatalog]
zpix = zcatalog/zpix-(sv1|sv2|sv3|main)-(backup|bright|dark|other)\.fits;zcatalog/zpix-SURVEY-PROGRAM.rst
ztile = zcatalog/ztile-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-(cumulative|pernight)\.fits;zcatalog/ztile-SURVEY-PROGRAM-TILETYPE.rst

[exclude]
calibnight = calibnight/[0-9]{8}/tmp/*;dummy
exposures = exposures/[0-9]{8}/old/*;dummy
preproc = preproc/[0-9]{8}/old/*;dummy
"""
    )


## Precompile Regular Expressions

In [4]:
r = dict()
structure = dict()
for s in parser.sections():
    r[s] = dict()
    structure[s] = dict()
    for key, value in parser.items(s):
        v = value.split(';')
        r[s][key] = re.compile(v[0])
        structure[s][key] = v[1]

## Scan the list of files

In [5]:
with open(fits_files) as e:
    data = e.readlines()
data.append(f'exposures-{specprod}.fits\n')
scanable = dict()
for file in data:
    ff = file.strip()
    f = ff.split('/')
    if len(f) == 1:
        section = 'top'
    elif f[1] == '1x_depth' or f[1] == '4x_depth':
        section = 'tiles:depth'
    else:
        section = f[0]
    if section not in scanable:
        scanable[section] = dict()
    excluded = False
    for key in r['exclude']:
        m = r['exclude'][key].match(ff)
        if m is not None:
            excluded = True
    if excluded:
        continue
    matched = False
    for key in r[section]:
        m = r[section][key].match(ff)
        if m is not None:
            matched = True
            if key in scanable[section]:
                scanable[section][key].append(ff)
            else:
                scanable[section][key] = [ff]
    if matched:
        continue
    print("ERROR: Could not match {0}!".format(ff))

## From the list of all file types, pick one at random

In [6]:
scan = dict()
for section in scanable:
    scan[section] = dict()
    for ftype in scanable[section]:
        scan[section][ftype] = choice(scanable[section][ftype])
scan

{'calibnight': {'fiberflatnight': 'calibnight/20201217/fiberflatnight-b7-20201217.fits',
  'psfnight': 'calibnight/20210613/psfnight-z8-20210613.fits'},
 'exposures': {'frame': 'exposures/20210223/00077755/frame-b7-00077755.fits',
  'psf': 'exposures/20210508/00087725/psf-b6-00087725.fits',
  'fiberflat': 'exposures/20210105/00070882/fiberflat-z9-00070882.fits',
  'sky': 'exposures/20210517/00089018/sky-r3-00089018.fits',
  'sframe': 'exposures/20210327/00082471/sframe-r9-00082471.fits',
  'cframe': 'exposures/20210223/00077777/cframe-z9-00077777.fits',
  'fluxcalib': 'exposures/20210106/00071111/fluxcalib-r2-00071111.fits',
  'stdstars': 'exposures/20210705/00097549/stdstars-0-00097549.fits',
  'shifted-input-psf': 'exposures/20210605/00091227/shifted-input-psf-b3-00091227.fits',
  'fit-psf': 'exposures/20210309/00079816/fit-psf-r7-00079816.fits',
  'fit-psf-before-blacklisted-fix': 'exposures/20210424/00086211/fit-psf-before-blacklisted-fix-z4-00086211.fits',
  'fit-psf-fixed-blackli

## Run fitsverify on the chosen files

In [ ]:
for section in scan:
    for key in scan[section]:
        command = ['fitsverify', '-l', scan[section][key]]
        proc = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE, cwd=specprod_path)
        out, err = proc.communicate()
        # print(section, key, out.decode('ascii').split('\n')[-2])
        result = out.decode('ascii').split('\n')[-2]
        if result == "**** Verification found 0 warning(s) and 0 error(s). ****":
            pass
            # print(section, key, "No problems detected.")
        else:
            print(section, key, "Problems detected!")
            print(out.decode('ascii'))
            if err:
                print(err.decode('ascii'))

## Generate datamodel stub files on all chosen files

In [7]:
for section in scan:
    for key in scan[section]:
        command = ['generate_model', os.path.join(specprod_path, scan[section][key])]
        print(' '.join(command))
        proc = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE)
        out, err = proc.communicate()
        if proc.returncode != 0:
            print(out.decode('ascii'))
            print(err.decode('ascii'))
        if key == 'shifted-input-psf':
            src = 'shifted'
        elif key == 'fit-psf' or key == 'fit-psf-fixed-blacklisted' or key == 'fit-psf-before-blacklisted-fix':
            src = 'fit'
        elif key == 'exposure-qa':
            src = 'exposure'
        elif key == 'tile-qa':
            src = 'tile'
        else:
            src = key
        if os.path.exists(f'{src}.rst'):
            d = os.path.dirname(structure[section][key])
            if d and not os.path.isdir(d):
                print("os.makedirs('{0}')".format(d))
                os.makedirs(d)
            print("shutil.move('{0}.rst', '{1}'".format(src, structure[section][key]))
            shutil.move('{0}.rst'.format(src), structure[section][key])
        else:
            print("ERROR: Could not find file corresponding to {0}.rst -> {1}!".format(src, scan[section][key]))
            

generate_model /global/cfs/cdirs/desi/spectro/redux/everest/calibnight/20201217/fiberflatnight-b7-20201217.fits
os.makedirs('calibnight/NIGHT')
shutil.move('fiberflatnight.rst', 'calibnight/NIGHT/fiberflatnight-CAMERA-EXPID.rst'
generate_model /global/cfs/cdirs/desi/spectro/redux/everest/calibnight/20210613/psfnight-z8-20210613.fits
shutil.move('psfnight.rst', 'calibnight/NIGHT/psfnight-CAMERA-EXPID.rst'
generate_model /global/cfs/cdirs/desi/spectro/redux/everest/exposures/20210223/00077755/frame-b7-00077755.fits
os.makedirs('exposures/NIGHT/EXPID')
shutil.move('frame.rst', 'exposures/NIGHT/EXPID/frame-CAMERA-EXPID.rst'
generate_model /global/cfs/cdirs/desi/spectro/redux/everest/exposures/20210508/00087725/psf-b6-00087725.fits
shutil.move('psf.rst', 'exposures/NIGHT/EXPID/psf-CAMERA-EXPID.rst'
generate_model /global/cfs/cdirs/desi/spectro/redux/everest/exposures/20210105/00070882/fiberflat-z9-00070882.fits
shutil.move('fiberflat.rst', 'exposures/NIGHT/EXPID/fiberflat-CAMERA-EXPID.rst'
